In [36]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from tqdm.notebook import tqdm
import math

In [37]:
C_p = 4186
A_container = 0.01
m_initial = 0.1
L_v = 2.26e6

def mpemba_cooling_model(t, y, T_env, h_base, h_exponent, K_evap):
    T, m = y[0], y[1]
    if T<= -5 or m <= 0 : return [0.0, 0.0]
    delta_T = np.maximum(T - T_env, 1e-6)
    h_convection = h_base * math.pow(delta_T, h_exponent)
    P_sat = 10**(8.07131 - (1730.63 / (233.426 + T))) if T > 0 else 0
    dm_dt = -K_evap * P_sat
    heat_loss_convection = h_convection * A_container * ( T - T_env )
    heat_loss_evaporation = -L_v * dm_dt
    total_heat_loss_rate = heat_loss_convection + heat_loss_evaporation
    dT_dt = -total_heat_loss_rate / (m * C_p)
    return [dT_dt, dm_dt]
    

In [38]:
def run_mpemba_simulation(T_initial, T_env, h_base, h_exponent, K_evap, time_limit=72000):
    y_initial = [T_initial, m_initial]
    sol = solve_ivp(
        fun=mpemba_cooling_model, t_span=[0, time_limit], y0=y_initial,
        args=(T_env, h_base, h_exponent, K_evap), dense_output=True
    )
    temperatures = sol.y[0]
    times = sol.t
    if np.all(temperatures > 0): return None
    zero_cross_indices = np.where(temperatures <= 0)[0]
    if len(zero_cross_indices) == 0: return None
    first_zero_cross_idx = zero_cross_indices[0]
    if first_zero_cross_idx == 0: return 0.0
    t1, T1 = times[first_zero_cross_idx-1], temperatures[first_zero_cross_idx-1]
    t2, T2 = times[first_zero_cross_idx], temperatures[first_zero_cross_idx]
    if T2 - T1 == 0: return t1
    return t1 + (0 - T1) * (t2 - t1) / (T2 - T1)

In [39]:
print("Simulation functions have been defined.")

Simulation functions have been defined.


In [40]:
num_samples = 4000
results = []

print(f"Running {num_samples} virtual experiments...")
for _ in tqdm(range(num_samples)):
    T_initial = np.random.uniform(5, 95)
    T_env = np.random.uniform(-20, -5)
    h_base = np.random.uniform(10, 50)
    h_exponent = np.random.uniform(0.0, 1.5)
    K_evap = np.random.uniform(1e-8, 1e-6)
    time_sec = run_mpemba_simulation(T_initial, T_env, h_base, h_exponent, K_evap)
    if time_sec is not None and time_sec > 0:
        results.append({
            'T_initial': T_initial, 'T_env': T_env, 'h_base': h_base,
            'h_exponent': h_exponent, 'K_evap': K_evap,
            'freezing_time': time_sec / 60 # convert to minutes
        })

df_mpemba = pd.DataFrame(results)
df_mpemba.to_csv('data/mpemba_data.csv', index=False)
print(f"\n Dataset with {len(df_mpemba)} valid samples saved to 'data/mpemba_data.csv'.")
df_mpemba.head()

Running 4000 virtual experiments...


  0%|          | 0/4000 [00:00<?, ?it/s]


 Dataset with 4000 valid samples saved to 'data/mpemba_data.csv'.


,T_initial,T_env,h_base,h_exponent,K_evap,freezing_time
0,9.501789,-11.955739,28.678967,0.228007,6.177706e-07,4.682257
1,18.418519,-6.220414,16.323292,0.688970,1.473262e-07,9.237274
2,77.322935,-14.696642,45.946000,1.429501,8.842334e-08,0.215854
3,54.065870,-15.431051,11.556988,1.074129,6.718782e-07,2.077791
4,63.550198,-17.476624,24.328732,1.096033,6.433669e-07,0.872834
